<a href="https://colab.research.google.com/github/Lenguist/insight-game-ai/blob/main/simple_sim_united.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Description moved to readme

# q learn

In [78]:
class Episode(object):
  def __init__(self, buyer, seller, verbose=True):
    self.buyer = buyer
    self.seller = seller
    self.verbose = verbose # whether to print info

  def negotiation_round(self):
    if self.verbose:
      print(f"Curr state: {self.seller.state}")

    offer = self.seller.make_offer()
    decision = self.buyer.check_offer(offer)
    new_state = {"last-offer":offer, "offers-made":self.seller.state["offers-made"]+1}
    self.seller.update_table(offer, decision, new_state)
    self.seller.state = new_state

    if self.verbose:
      print(f"Seller made offer of {offer}. The buyer decided to {decision}")
    return decision

  def run_episode(self, episode_num):
    decision = ""
    while decision != "walk away" and decision != "accept offer":
      decision = self.negotiation_round()

    if decision == "accept offer":
      final_offer = self.seller.state["last-offer"]
      profit = final_offer - self.seller.value
      if self.verbose:
        print(f"Deal made at {final_offer}")
        print(f"Buyer's max_price was {self.buyer.maxprice}")

    else:
      if self.verbose:
        print(f"No deal made - no profit.")
        print(f"Buyer's max_price was {self.buyer.maxprice}")
      profit = 0
    # reset seller state
    self.seller.reset_state(episode_num)
    return profit

In [181]:
"Seller classes"
import random
import numpy as np
import math

class Seller(object):
  def __init__(self, value, init_offer):
    self.value = value
    self.init_offer = init_offer
    self.state = {"last-offer": self.init_offer, "offers-made":0} # initial state
  def make_offer(self):
    raise NotImplementedError("Subclasses should implement this method.")
  def reset_state(self):
    self.state = {"last-offer":self.init_offer, "offers-made":0}
  def update_table(self):
    pass
  
class QLearningSeller(Seller):
  def __init__(self, value, init_offer):
    super().__init__(value, init_offer)
    action_space_size = self.init_offer - self.value
    state_space_size = (self.init_offer - self.value) ** 2
    self.q_table = np.zeros((state_space_size, action_space_size))

    # Q-Learning parameters
    self.learning_rate = 0.1
    self.discount_rate = 0.99
    self.exploration_rate = 1
    self.max_exploration_rate = 1
    self.min_exploration_rate = 0.01
    self.exploration_decay_rate = 0.001

  def reset_state(self,episode_num):
    super().reset_state()
    #self.exploration_rate = self.min_exploration_rate + (self.max_exploration_rate - self.min_exploration_rate) * np.exp(-self.exploration_decay_rate*episode_num)
    self.exploration_rate = 0.1                                                                 
    # exploration_rate = self.min_exploration_rate + \
    # (self.max_exploration_rate - self.min_exploration_rate) * np.exp(-self.exploration_decay_rate*self.episode)

  
  def make_offer(self):
    random_number = random.uniform(0, 1)
    if random_number > self.exploration_rate:
      offer = np.argmax(self.q_table[self.get_state_array_number(self.state), :]) + (self.value+1)
    else:
      offer = random.randint(self.value+1,self.state['last-offer']-1)
    return offer
  
  # helper methods
  def get_state_array_number(self, state):
    return state['last-offer']-(self.value+1)+(self.init_offer-self.value)*state['offers-made']

  def from_array_num_get_state(self, array_num):
    offers_made = math.floor(array_num/(self.init_offer-self.value))
    last_offer =  array_num % (self.init_offer-self.value) + (self.value + 1)
    return {"last-offer":last_offer, "offers-made":offers_made}


  def update_table(self, offer, decision, new_state):
    if decision == 'accept offer':
      reward = offer - self.value
    else:
      reward = 0
    
    state_array_number = self.get_state_array_number(self.state)
    new_state_array_number = self.get_state_array_number(new_state)
    self.q_table[state_array_number, offer-(self.value+1)] = self.q_table[state_array_number, offer-(self.value+1)] * (1 - self.learning_rate) + \
                                  self.learning_rate * (reward + self.discount_rate * np.max(self.q_table[new_state_array_number, :]))



In [184]:
# imp_incr constant, max_price random
def basic_simulation(value, range_min, range_max, imp_incr, rounds, seller, verbose=False):
  total_profit = 0
  profit_tracker = []
  round_cnt = 0
  for i in range(rounds):
    maxprice = int(random.uniform(range_min, range_max+1))
    buyer = Buyer(maxprice, 0, imp_incr)
    episode = Episode(buyer, seller, verbose=False)
    profit = episode.run_episode(i)
    total_profit += profit

    # profit_tracker.append(profit)
  average_return = total_profit/rounds

  # # profit tracker is an array containing the profit for each round
  # count=0
  # print(f"profit_tracker: {profit_tracker}")
  # list_of_ave_profit_per_thousand = []
  # profit_per_thousand = 0
  # for p in profit_tracker:
  #   if(count<1000):
  #     print(f'profit_per_thousand: {profit_per_thousand}')
  #     print(f'p: {p}')
  #     profit_per_thousand+=p
  #     count+=1
  #   else:
  #     list_of_ave_profit_per_thousand.append(profit_per_thousand)
  #     profit_per_thousand = 0
  #     count=0
  # print(list_of_ave_profit_per_thousand)

  return average_return

In [185]:
from buyer import Buyer
# params that are needed to initialize seller
value = 10
range_min = 11
range_max = 30
rounds = 50000

init_offer=range_max

seller = QLearningSeller(value=value, init_offer=init_offer)

for i in range(10):    
result = basic_simulation(value,
                        range_min,
                        range_max,
                        imp_incr=  0.1,
                        rounds = rounds,
                        seller=seller,
                        verbose=False)
print(result)
print(seller.q_table)

6.97184
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from buyer import Buyer
# params that are needed to initialize seller
value = 10
range_min = 11
range_max = 30
rounds = 500

init_offer=range_max

seller = QLearningSeller(value=value, init_offer=init_offer)
rewards_all_episodes = []
for i in range(10):
    result = basic_simulation(value,
                            range_min,
                            range_max,
                            imp_incr=  0.1,
                            rounds = rounds,
                            seller=seller,
                            verbose=False)
    rewards_all_episodes.append(result)
    print(seller.q_table)
    print(rewards_all_episodes)



[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[5.532]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[5.532, 5.412]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[5.532, 5.412, 5.47]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[5.532, 5.412, 5.47, 5.52]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[5.532, 5.412, 5.47, 5.52, 5.648]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0.

In [186]:
import pandas as pd
q = pd.DataFrame(seller.q_table)

In [187]:
q

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
import pandas as pd
import numpy as np



# Filtering dataframe to get only non-zero rows
non_zero_df = q.loc[(q!=0).any(axis=1)]

print(non_zero_df)


           0         1         2         3         4         5         6   \
19   0.994274  2.070548  3.059700  4.148997  4.604474  5.400382  4.773324   
21   0.190000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
22   0.100000  0.377992  0.000000  0.000000  0.000000  0.000000  0.000000   
23   0.100000  0.765257  0.000000  0.000000  0.000000  0.000000  0.000000   
24   0.000000  0.614338  0.105770  0.000000  0.000000  0.000000  0.000000   
25   0.000000  0.197820  1.552129  0.058292  0.134355  0.000000  0.000000   
26   0.190000  0.000000  0.748400  0.384121  2.415127  0.079758  0.000000   
27   0.530579  1.080250  2.690339  1.605084  2.350428  1.686395  1.872535   
28   0.271000  0.461606  0.306328  0.411508  1.238429  2.976569  0.667132   
29   0.409510  0.000000  0.654793  0.824448  2.578491  0.717381  0.844345   
30   0.100000  0.914373  0.039482  2.642073  1.074965  0.905066  1.492662   
31   0.336610  0.991771  1.226595  0.839714  0.855699  2.446038  4.360484   